In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
import datetime

In [1]:
#Tee Times come in format of '6:39am\n18\n4' only grabbing AM so just focus on numbers. 
def cleanTime(tee_times):
    sep = "a"
    clean_AM = []
    for tee in tee_times:
        clean_AM.append(tee.split(sep, 1)[0])
    return clean_AM

In [4]:
#Only interested in AM tee times. 
def findAM(tee_times):
    subs = 'am'
    times_AM = list(filter(lambda x: subs in x, tee_times))
    return times_AM 

In [5]:
#find best tee time only 8 or 9 AM times. First one will be booked. 
def bestTime(times_AM):
    tee_eight = list(filter(lambda x: '8' in x, times_AM))
    if tee_eight == []:
        tee_nine = list(filter(lambda x: '9' in x, times_AM))
        if tee_nine == []:
            print("NO GOOD TIMES FOR YOU")
        else: 
            my_time = tee_nine[0]
            return my_time
    else: 
        my_time = tee_eight[0]
        return my_time

In [6]:
#Write a function to find the next Saturday
def getSaturday():
    d=datetime.date.today()
    d.day
    if d.weekday() == 5:
        d += datetime.timedelta(7)
        print('next saturday is',d.day)
    else:
        while d.weekday() != 5:
            d += datetime.timedelta(1)
    return d

In [7]:
#Do not post sensittive data to internet. Keep info in local file 
data = pd.read_csv('info.txt')
username = data['Login Info'][0]
password= data['Login Info'][1]
V_WEBSITE = data['Login Info'][2]

In [8]:
#I like to golf on Saturday. Find next saturday 
day_saturday = getSaturday()
sat_string = day_saturday.strftime("%m-%d-%Y")
sat_string

next saturday is 5


'06-05-2021'

In [9]:
#create an instance of firefox using selenium. Sleep to allow load times
driver = webdriver.Firefox()
time.sleep(2)

In [10]:
#Maximize the window. When window is small some of the text boxes change to drop downs
driver.maximize_window()

In [11]:
#open the website
driver.get(V_WEBSITE)
time.sleep(2)

In [12]:
#Find the xpath of the first button needed for log-in
resident_xpath = '//*[@id="content"]/div/button[1]'
driver.find_element_by_xpath(resident_xpath).click()
time.sleep(2)

In [13]:
#click log-in
driver.find_element_by_class_name('login').click()

In [14]:
#find email field and fill in 
email_box = driver.find_element_by_id('login_email')
email_box.send_keys(username)

In [15]:
#find password field and fill in 
pass_box = driver.find_element_by_id('login_password')
pass_box.send_keys(password)

In [16]:
#find CSS selector for submit button. Note: Xpath was messy 
login_CSS=driver.find_element_by_css_selector('.col-md-2')
login_CSS.click()
time.sleep(2)

In [17]:
#now we are logged in and at the profile page. There is only one button on display
#use this button to get to tee time reservations 
reserve_CSS=driver.find_element_by_css_selector('.btn')
reserve_CSS.click()

In [18]:
#allow time to load
time.sleep(2)

In [19]:
#use the dropdown menu to select the course that we are looking to play 
el = driver.find_element_by_id('schedule_select')
for option in el.find_elements_by_tag_name('option'):
    if option.text == 'Bethpage Blue Course':
        #print(option.text)
        option.click() # select() in earlier versions of webdriver
        break


In [20]:
#check off that we have 4 players
players_CSS=driver.find_element_by_css_selector('a.btn:nth-child(4)')
players_CSS.click()

In [21]:
#enter the date that we want to play by deleting the current date
#enter satureday string
date_box = driver.find_element_by_id('date-field')
time.sleep(2)
date_box.send_keys(Keys.CONTROL + "a")
date_box.send_keys(Keys.DELETE)
date_box.send_keys(sat_string)
date_box.send_keys(Keys.ENTER)
time.sleep(2)

In [21]:
#get all the tee times 
elements = driver.find_elements_by_class_name('reserve-time')
tee_times = []
for e in elements:
    tee_times.append(e.text)

In [22]:
#focus on AM tee times
time_AM = findAM(tee_times)
time_AM

['6:39am\n18\n4',
 '6:57am\n18\n4',
 '8:18am\n18\n4',
 '9:39am\n18\n4',
 '10:51am\n18\n4']

In [23]:
#get it in readable format 
clean_AM = cleanTime(time_AM)

In [24]:
# what time is best for you, 8-9
my_time = bestTime(clean_AM)
my_time

'8:18'

In [26]:
#find the index of the element so we know what to click on 
x_index = [s for s in tee_times if my_time in s]
index_e = tee_times.index(x_index[0])
index_e

2

In [27]:
#click on the tee time 
my_element = elements[index_e]
my_element.click()
time.sleep(1)

In [ ]:
#book the tee time
book_xpath=driver.find_element_by_css_selector('.btn-success')
book_xpath.click()